In [1]:
from src.tools_dev import Mapdisplay, random_point_in_shp
import ee
import tensorflow as tf
import geopandas as gpd
from shapely.geometry import Point, Polygon
from folium.plugins import MarkerCluster
import numpy as np
from src.tools_dev import get_bound_points, generate_box_buffer, \
                            generate_ee_points, get_landsat_data
import geopy.distance

import folium
print('Folium version: ' + folium.__version__)

# Define the URL format used for Earth Engine generated map tiles.
EE_TILES = 'https://earthengine.googleapis.com/map/{mapid}/{{z}}/{{x}}/{{y}}?token={token}'
ee.Initialize()
%load_ext autoreload
%autoreload 2
%matplotlib inline

Folium version: 0.11.0


In [2]:
BUCKET = 'ikh-nart'
shp = './data/shp/IN_boundary/IN_boundary.shp'

#BUCKET = 'rio-mora'
#shp = './data/shp/RioMoraNWRBndry_2019/RioMoraNWRBndry_2019.shp'
#shp = './data/shp/MRW/Mora River Watershed July2011.shp'

In [3]:
#bbox = gpd.read_file(shp)
bbox = gpd.read_file(shp)
bbox = bbox.to_crs("EPSG:4326")
xmin, ymin, xmax, ymax = bbox.bounds.values[0]
# Passing a rectangle (prediction area) to Earth Engine
roi = ee.Geometry.Rectangle([xmin,ymin,xmax,ymax])
center = roi.centroid().getInfo()['coordinates']
center.reverse()

In [4]:
#bottom edge
lower_edge = geopy.distance.geodesic((ymin,xmin), (ymin,xmax)).meters
#top edge
upper_edge = geopy.distance.geodesic((ymax,xmin), (ymax,xmax)).meters
#left edge
left_edge = geopy.distance.geodesic((ymin,xmin), (ymax,xmin)).meters
#right edge
right_edge = geopy.distance.geodesic((ymin,xmax), (ymax,xmax)).meters
#the largest are the left and right edges
buffer = int(np.ceil(left_edge))

In [5]:
#use the centroid as the single point
#points = gpd.read_file('./data/sample_points.csv')
points = gpd.GeoDataFrame(np.array(center).reshape((len(center),1)).T, columns=['Latitude', 'Longitude'])
points.Latitude = points.Latitude.astype('float')
points.Longitude= points.Longitude.astype('float')
geometry = [Point(xy) for xy in zip(points.Longitude, points.Latitude)]
crs = {'init': 'epsg:4326'} #http://www.spatialreference.org/ref/epsg/2263/
geo_df = gpd.GeoDataFrame(points, crs=crs, geometry=geometry)
geo_df['id'] = BUCKET

/usr/local/lib/python3.6/dist-packages/pyproj/crs/crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [6]:
map_ex = Mapdisplay(center,{BUCKET:roi.getInfo()},zoom_start=9)
[folium.Marker((p.geometry.y,p.geometry.x),popup=p.id).add_to(map_ex) \
     for _,p in geo_df.iterrows()]
map_ex

In [7]:
#need to figure out a way to make a buffer that encompasses the entire
#point. We know the bounding box. Can we measure the length of the bounding
#box in meters?
extra = 1000 #add an extra kilometer buffer
BUFFER = (buffer+extra)/2
p = generate_ee_points(geo_df)
boxes = generate_box_buffer(geo_df, BUFFER)
fc_boxes = boxes['ee_fc']
box = boxes['box_df']

training_list = fc_boxes.toList(fc_boxes.size())
training_pnts = p.toList(p.size())

In [8]:
 #need to understand the format of these geometries
polyImage = ee.Image(0).byte().paint(fc_boxes, 1).paint(fc_boxes, 2)
polyImage = polyImage.updateMask(polyImage)

mapid = polyImage.getMapId({'min': 1, 'max': 2, 'palette': ['red', 'blue']})

#map_ex = Mapdisplay(center,{'Truckee':truckee.getInfo()},zoom_start=9)
map_ex = folium.Map(center,zoom_start=10)
tile = folium.TileLayer(
        #tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        tiles = 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png',
        attr = 'Esri',
        name = 'Esri Satellite',
        overlay = False,
        control = True
       ).add_to(map_ex)
#add points
[folium.Marker((p.geometry.y,p.geometry.x),popup=p.id).add_to(map_ex) \
     for _,p in geo_df.iterrows()]
#add boxes
[folium.GeoJson(geom).add_to(map_ex) for geom in box.geometry]
map_ex

In [11]:
landsat_annual = []
startyear = 1986
endyear = 2021 
max_cloud_thr = 20
for year in range(startyear, endyear):
    landsat = get_landsat_data(fc_boxes, datelist=[year,1,1], years_back=0,\
                               max_cloud_thr=max_cloud_thr, \
                               bands=['B1', 'B2', 'B3', 'B4', 'B5', 'B7'],\
                               month_aggregate=True, verbose=True)
    landsat_annual.append(landsat)
    
years = np.arange(startyear,endyear)

Found 7 LANDSAT images from 01-01-1986 to 12-31-1986.
|  Month number	|  Image count 	|
|	 1 	|	 0 	|
|	 2 	|	 0 	|
|	 3 	|	 0 	|
|	 4 	|	 0 	|
|	 5 	|	 0 	|
|	 6 	|	 2 	|
|	 7 	|	 2 	|
|	 8 	|	 0 	|
|	 9 	|	 0 	|
|	 10 	|	 0 	|
|	 11 	|	 3 	|
|	 12 	|	 0 	|

Collected LANDSAT data and aggregated to single image with 18 bands represent mean month for bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'].

Warning!! LANDSAT data missing, only 18 bands. Check date range!

Found 13 LANDSAT images from 01-01-1987 to 12-31-1987.
|  Month number	|  Image count 	|
|	 1 	|	 1 	|
|	 2 	|	 1 	|
|	 3 	|	 1 	|
|	 4 	|	 0 	|
|	 5 	|	 2 	|
|	 6 	|	 1 	|
|	 7 	|	 1 	|
|	 8 	|	 0 	|
|	 9 	|	 2 	|
|	 10 	|	 1 	|
|	 11 	|	 3 	|
|	 12 	|	 0 	|

Collected LANDSAT data and aggregated to single image with 54 bands represent mean month for bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'].

Warning!! LANDSAT data missing, only 54 bands. Check date range!

Found 12 LANDSAT images from 01-01-1988 to 12-31-1988.
|  Month numbe

|	 5 	|	 3 	|
|	 6 	|	 2 	|
|	 7 	|	 3 	|
|	 8 	|	 2 	|
|	 9 	|	 3 	|
|	 10 	|	 3 	|
|	 11 	|	 6 	|
|	 12 	|	 5 	|

Collected LANDSAT data and aggregated to single image with 72 bands represent mean month for bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'].

Found 40 LANDSAT images from 01-01-2005 to 12-31-2005.
|  Month number	|  Image count 	|
|	 1 	|	 3 	|
|	 2 	|	 2 	|
|	 3 	|	 5 	|
|	 4 	|	 5 	|
|	 5 	|	 3 	|
|	 6 	|	 4 	|
|	 7 	|	 2 	|
|	 8 	|	 3 	|
|	 9 	|	 5 	|
|	 10 	|	 5 	|
|	 11 	|	 3 	|
|	 12 	|	 0 	|

Collected LANDSAT data and aggregated to single image with 66 bands represent mean month for bands: ['B1', 'B2', 'B3', 'B4', 'B5', 'B7'].

Warning!! LANDSAT data missing, only 66 bands. Check date range!

Found 30 LANDSAT images from 01-01-2006 to 12-31-2006.
|  Month number	|  Image count 	|
|	 1 	|	 1 	|
|	 2 	|	 3 	|
|	 3 	|	 0 	|
|	 4 	|	 2 	|
|	 5 	|	 4 	|
|	 6 	|	 4 	|
|	 7 	|	 2 	|
|	 8 	|	 3 	|
|	 9 	|	 1 	|
|	 10 	|	 3 	|
|	 11 	|	 4 	|
|	 12 	|	 3 	|

Collected LANDSAT 

In [12]:
FOLDER = 'tif_output'
REGION = fc_boxes.first().geometry().bounds().getInfo()['coordinates']
for img,year in zip(landsat_annual,years):
    desc = f'landsat_{year}'
    selector_bands = [b['id'] for b in img.getInfo()['bands']]
    task = ee.batch.Export.image.toCloudStorage(image=img,\
                                description=desc,\
                                bucket=BUCKET,\
                                fileNamePrefix=f'{FOLDER}/{desc}',\
                                fileFormat='GeoTIFF',\
                                scale=30,\
                                region=REGION)
    task.start()

In [57]:
#landsat is an image collection, which we can filter by both year and month
#and aggregate by the month of year
#there are currently 84 bands because there ar 7 bands by 12 months
#we should then have n-year images that are 84 bands to get monthly averages by year
#could run get_landsat_data with an iterator on the datelist 

In [101]:
#we can also just export each of these images as tiffs.
#this is the easiest right now

{'type': 'Image',
 'bands': [{'id': 'B1',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B2',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B3',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B4',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B5',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'B7',
   'data_type': {'type': 'PixelType', 'precision': 'float'},
   'crs': 'EPSG:4326',
   'crs_transform': [1, 0, 0, 0, 1, 0]},
  {'id': 'cloud',
   'data_type': {'type': 'PixelType',
    'precision': 'double',
    'min': 0,
    'max': 100},
   'crs': 'EPSG:432

In [ ]:
#need to aggregate the landsat by month and year